In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import pandas as pd
from PIL import Image

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
class_names = ["0", "1", "2", "3", "4", "5"]

def load_image(img_path, img_size=(256, 256)):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array, img


def predict_image(model, img_array):
    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions, axis=1)[0]
    predicted_class = class_names[predicted_index]
    probabilities = predictions[0]
    return predicted_class, probabilities

In [ ]:
model_path = "./best_model.keras"

model = tf.keras.models.load_model(model_path)

In [ ]:
inference_images_path = './inference_images'

image_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
image_filenames = [f for f in os.listdir(inference_images_path) if f.lower().endswith(image_formats)]
image_paths = [os.path.join(inference_images_path, f) for f in image_filenames]

print(f"Found {len(image_paths)} images")

In [ ]:
results = []

for img_path in image_paths:
    img_array, img = load_image(img_path)

    predicted_class_name, probabilities = predict_image(model, img_array) 

    results.append({
        'filename': os.path.basename(img_path),
        'predicted_class': predicted_class_name,
        'probabilities': probabilities
    })

df_results = pd.DataFrame(results)

probabilities_df = pd.DataFrame(df_results['probabilities'].tolist(), columns=[f'prob_{cls}' for cls in class_names])
df_results = pd.concat([df_results.drop('probabilities', axis=1), probabilities_df], axis=1)

df_results.head()